# モデルの検証
実際の楽曲データをモデルで分類する

サンプル曲（全18曲）
* サンプル曲のジャンルラベル付は検証者が恣意的に行なった
* PopのサンプルにはGTZANには含まれていないJ-POP, K-POPを含めた
* Classicalのサンプルにはインストルメンタルと音声入りを一つずつ用いた
* Countryのサンプルは含まれていない
* GTZAN分類に含まれていないボサノバの曲も検証した

In [1]:
import math
import json
import os
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa  # https://librosa.org/doc/latest/index.html
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
base_path = '/content/drive/My Drive/Colab Notebooks/Music-Genre-Classification'
sample_data_path = os.path.join(base_path, 'Data', 'sample')

In [5]:
# Load models
GRU = load_model(os.path.join(base_path, 'GTZAN_128GRU.keras'))
Conv1D = load_model(os.path.join(base_path, 'GTZAN_Conv1D_BN.keras'))

In [6]:
genres = ['Blues', 'Classical', 'Country', 'Disco', 'Hip hop', 'Jazz', 'Metal', 'Pop', 'Reggae', 'Rock']

In [7]:
# Extract MFCC
def extract_mfcc(file_path, n_mfcc=13, n_fft=2048, hop_length=512, sample_rate=22050):
    audio, sr = librosa.load(file_path, sr=sample_rate)
    # Pass 'audio' as a keyword argument to 'mfcc' function
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T
    return mfcc

# Shape MFCC features into (130, 13)
def pad_mfcc(mfcc, pad_width=130, n_mfcc=13):
    current_length = mfcc.shape[0]
    if current_length > pad_width:
        mfcc = mfcc[:pad_width]
    elif current_length < pad_width:
        padding = np.zeros((pad_width - current_length, n_mfcc))
        mfcc = np.vstack((mfcc, padding))
    return mfcc

In [8]:
# Classify genres of sample data
def classify_genres(dir_path, genres, model):
    results = {}
    for filename in os.listdir(dir_path):
        if filename.endswith(".wav"):
            file_path = os.path.join(dir_path, filename)
            mfcc_features = extract_mfcc(file_path)
            mfcc_features = pad_mfcc(mfcc_features)
            mfcc_features = mfcc_features[np.newaxis, ...]
            predictions = model.predict(mfcc_features)
            predicted_genre = np.argmax(predictions)
            results[filename] = predicted_genre
    return results

## 128-Unit GRU Modelでの検証

In [9]:
# Classify using GRU Model
genre_predictions = classify_genres(sample_data_path, genres, GRU)
for filename, i in genre_predictions.items():
    print(f"File: {filename} - Predicted Genre: {genres[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
File: Disco_DontStopTilYouGetEnough(MichaelJackson).wav - Predicted Genre: Rock
File: Classic_MinuteWaltz(Chopin).wav - Predicted Genre: Classical
File: Classic_Op60OnTheWater(Voice).wav - Predicted Genre: Classical
File: TheGirlFromIpanema(AstrudGilberto).wav - Predicted Genre: Pop
File: Pop_Lemon(KenshiYonezu).w

## Conv1D with Batch Normalization Modelでの検証

In [10]:
# Classify using Conv1D Model
genre_predictions = classify_genres(sample_data_path, genres, Conv1D)
for filename, i in genre_predictions.items():
    print(f"File: {filename} - Predicted Genre: {genres[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
File: Disco_DontStopTilYouGetEnough(MichaelJackson).wav - Predicted Genre: Hip hop
File: Classic_MinuteWaltz(Chopin).wav - Predicted Genre: Classical
File: Classic_Op60OnTheWater(Voice).wav - Predicted Genre: Classical
File: TheGirlFromIpanema(AstrudGilberto).wav - Predicted Genre: Country
File: Pop_Lemon(KenshiYo

## 予測結果

| 曲名                          | 実際のジャンル | 128GRU予測ジャンル | 128GRU正誤 | Conv1D_BN予測ジャンル | Conv1D_BN正誤 |
|-------------------------------|--------------|-------------------|------------|----------------------|---------------|
| Minute Waltz                   | Classic      | Classical         | 正解       | Classical            | 正解          |
| Op. 60 On The Water            | Classic      | Classical         | 正解       | Classical            | 正解          |
| The Girl From Ipanema          | Bossa        | Pop               | 誤り       | Country              | 誤り          |
| Desafinado                     | Bossa        | Jazz              | 誤り       | Jazz                 | 誤り          |
| Cry Me A River                 | Jazz         | Jazz              | 正解       | Rock                 | 誤り          |
| Warm Up And Dialogue Between Lee And Rudy | Jazz | Jazz           | 正解       | Jazz                 | 正解          |
| Bling Bang Bang Born           | Hip hop      | Hip hop           | 正解       | Hip hop              | 正解          |
| Redemption Song                | Reggae       | Jazz              | 誤り       | Hip hop              | 誤り          |
| You'll Never Find              | Reggae       | Jazz              | 誤り       | Hip hop              | 誤り          |
| Lemon                          | Pop          | Classical         | 誤り       | Hip hop              | 誤り          |
| Ice Cream                      | Pop          | Hip hop           | 誤り       | Hip hop              | 誤り          |
| All I Want                     | Pop          | Jazz              | 誤り       | Reggae               | 誤り          |
| Idle                           | Pop          | Pop               | 正解       | Pop                  | 正解          |
| Psychosocial                   | Metal        | Hip hop           | 誤り       | Jazz                 | 誤り          |
| Stand And Deliver              | Metal        | Country           | 誤り       | Country              | 誤り          |
| Smells Like Teen Spirit        | Rock         | Reggae            | 誤り       | Jazz                 | 誤り          |
| Vor Der Schlacht               | Rock         | Disco             | 誤り       | Disco                | 誤り          |
| Don't Stop 'Til You Get Enough | Disco        | Rock              | 誤り       | Hip hop              | 誤り          |


考察

* クラシックについては両モデルが正しく分類したが、そのほかのジャンルは正確な分類が難しかった
* またクラシックに誤分類された楽曲もなかったのでクラシック分類は比較的容易と言える
* GRUモデルはジャズへの分類に偏り、Conv1Dモデルはヒップホップへの分類に偏っている
* GRUモデルはジャズサンプルを2曲とも正しく分類した。しかし前述の通りレゲエやポップスもジャズと分類されているため、ジャズでよく用いられる曲構成が汎化されてしまった可能性がある
* ポップスについては、そのジャンルに含まれる音楽の多様性が幅広く、また検証者によるラベル付も恣意的であったことから正誤を評価することは難しいが、GRUモデルではクラシックと分類されたLemonがConv1Dモデルではヒップホップと音楽的にまったく異なるジャンルに分類されたのは興味深い
* レゲエ2曲についてはどちらのモデルも同じジャンルに誤分類したが、GRUはジャズ、Conv1Dはヒップホップとそれぞれが分類しがちなジャンルであった
* 一方で人間の耳には絶対にレゲエではないサンプル曲がそれぞれのモデルで一つずつレゲエに分類されていることから、本来のレゲエっぽさとは関係のない特徴がGTZANのレゲエデータセットから抽出された可能性がある
* GTZANデータセットに用意されていなかったボサノバについては、Desafinadoのジャズ分類は正解と言えるだろう。イパネマについては、ポップスに分類したGRUに軍配か